# Outline
Text goes here

In [3]:
# Imports
import pandas as pd
from citipy import citipy
import random

# Get API key from api_keys.py
import api_keys
api_ow = api_keys.api_ow

In [4]:
# Sample API call for OpenWeather:
# http://api.openweathermap.org/geo/1.0/direct?q=CITYNAME,STATECODE,COUNTRYCODE&limit=LIMIT&appid=APIKEY

In [5]:
# Functions

# Generates a random number between -180 and 180 degrees (0 is Greenwich, England).
def genLatitude():
    randomLat = random.uniform(-180, 180)
    return round(randomLat, 2)

# Generates a random number between -90 and 90 degrees (0 is equator)
def genLongitude():
    randomLong = random.uniform(-90, 90)
    return round(randomLong, 2)

In [19]:
# Dataframe to hold city data.
df_cities = pd.DataFrame(columns=["City Name", "Latitude", "Longitude", "Temperature (F)", "Humidity (%)", "Cloudiness (%)", "Wind speed (MPH)"])
df_cities

,City Name,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind speed (MPH)


In [14]:
# Generate 500 random sets of long/lat coordinates
# Get the nearest city to each coordinates and add its weather data if it's unique
cities_500 = []
count = 1
while count <= 500:
    long = genLongitude()
    lat = genLatitude()
    nearest = citipy.nearest_city(lat, long)
    # Only add the city if it's unique
    if nearest not in cities_500:
        cities_500.append(nearest.city_name)
        count += 1

In [18]:
# For each city, get its exact coordinates in the OpenWeather API
# This is a little backwards, but it's the most straightforward way to prevent an API call to broken coordinates
for city in cities_500:
    print(city[0])

g
k
n
g
t
g
l
o
t
k
i
p
u
a
t
v
j
m
c
c
t
i
a
s
g
g
g
a
p
g
v
q
p
y
l
v
r
d
l
p
s
l
l
g
h
l
c
l
h
l
p
i
o
u
m
l
d
p
d
a
i
g
q
m
r
i
q
h
s
p
t
g
u
o
i
m
l
o
a
e
p
i
g
o
k
b
a
t
g
p
o
t
v
l
u
p
d
p
p
g
g
k
o
u
p
g
d
i
a
p
p
h
k
i
h
t
o
i
i
g
l
h
i
g
g
h
u
p
h
e
l
c
y
s
l
l
a
l
m
u
t
h
u
g
p
g
p
l
g
t
q
m
r
u
e
i
h
i
u
p
p
o
l
p
v
p
l
g
e
l
p
p
i
u
v
p
j
e
g
f
d
i
q
r
i
e
g
c
s
u
u
y
p
g
i
o
v
p
i
b
v
d
l
g
l
i
p
g
u
g
g
i
t
i
e
r
u
k
p
g
p
p
l
e
v
l
t
j
p
g
i
a
p
g
m
v
p
l
i
r
i
g
p
n
u
l
i
m
l
g
a
d
p
l
v
s
b
y
p
i
u
m
s
g
d
g
a
n
q
d
r
i
n
s
p
l
t
g
e
i
u
d
g
p
g
i
t
p
u
p
p
l
b
p
i
l
p
w
l
l
g
v
l
l
a
l
l
o
g
m
p
h
p
u
g
o
m
l
g
s
r
i
g
p
t
k
p
g
v
o
d
p
g
p
i
p
g
s
q
g
p
p
p
p
r
a
v
s
l
i
l
p
l
d
t
l
i
p
g
b
a
m
l
a
s
g
v
c
a
m
d
t
p
b
p
i
h
d
o
l
m
p
l
k
s
i
u
p
p
t
o
b
g
g
o
h
i
w
n
f
m
u
d
p
p
u
i
p
d
u
g
a
g
l
p
i
p
p
l
e
i
t
g
p
a
u
c
g
p
p
d
s
g
v
g
g
p
p
p
i
l
n
d
l
p
p
k
g
a
g
g
p
i
g
i
m
p
l
p
u
l
a
t
s
v
a
i
i
m
l
l
c
l
t
s
l
p
i
c
u
c
l
s
r
i
t
p
w
l
h
s
t
p
u
t
p
i
i
p
n


In [ ]:
# For each city, get its temperature, humidity, cloudiness, and wind speed
# Save the data into a dataframe

In [16]:
cities_500

['guanajay',
 'kegen',
 "nar'yan-mar",
 'grytviken',
 'taltal',
 'georgetown',
 'loukhi',
 'olonkinbyen',
 'talnakh',
 'korla',
 'iskateley',
 'port mathurin',
 'ushuaia',
 'aasiaat',
 'tazovsky',
 'vila franca do campo',
 'juigalpa',
 'mbaiki',
 'clinton',
 'cromer',
 'tokar',
 'isafjordur',
 'al bardiyah',
 'sinabang',
 'grytviken',
 'grytviken',
 'grytviken',
 'aasiaat',
 'port-aux-francais',
 'grytviken',
 'vorgashor',
 'qaqortoq',
 'port-aux-francais',
 'yirol',
 'longyearbyen',
 'vila do maio',
 'raja',
 'dudinka',
 'longyearbyen',
 'port-aux-francais',
 'stanley',
 'longyearbyen',
 'longyearbyen',
 'grytviken',
 'hermanus',
 'longyearbyen',
 'constantia',
 'longyearbyen',
 'hamilton',
 'longyearbyen',
 'port-aux-francais',
 'ilulissat',
 'ondjiva',
 'ushuaia',
 'mwene-ditu',
 'longyearbyen',
 'dudinka',
 'port-aux-francais',
 'dudinka',
 'ati',
 'ilulissat',
 'grytviken',
 'quellon',
 'murzuq',
 'ribeira grande',
 'ilulissat',
 'qarazhal',
 'hadibu',
 'sinabang',
 'port elizabet